In [1]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel #check this method and properties
from openqaoa.backends import create_device

# method to find the correct states for the QAOA object 
from openqaoa.utilities import ground_state_hamiltonian

## El problema de la mochila 

El problema de la mochila es un problema combinatorio que considera un conjunto de objetos que tiene un valor peso/costo, el objetivo consiste en seleccionar un subconjunto de elementos que permita maximizar el peso/costo. Este problema esta restringido a una capacidad máxima específica.

Para ello se debe cumplir que el peso máximo debe ser mayor al menor de los pesos de los elementos, en el caso trivial la suma de todos los elementos es menor al peso máximo.

Para la transformación a un problema QUBO se deben considerar las restricciones de la desigualdad. 

La función puede describirse fomo:

$$

max \left(  \sum_{i=0}^n v_i x_i \right) 

$$

La restricción es:

$$

\sum_{i=1}^n w_i x_i \leq W
$$

$$

x_i = \{0,1\}

$$

### Problema de optimización

Existe una gran variedad de problemas que pueden resolverde con ese planteamiento, por ejemplo:
-Si se desea maximizar la producción agricola en un invernadero dados la cantidad de sustratos y el costo de los productos a coshechar.
-Se desea maximizar la contratación de profesores dado un presupuesto definido que permita ofrecer la mayor cantidad de áreas de conocimiento.
-Para maximizar el almacenamiento de productos para su translado, la etensión de este caso se proyecta al uso de múltiples mochilas en donde se pueden almacenar, la misma o diferentes cantidad de peso.


In [2]:
# Datos del problema de Prueba
values = np.random.randint(10, size=10)
weights = np.random.randint(10, size=10)
max_weight = np.random.randint(10,50)

print(values)
print(weights)
print(max_weight)


[2 5 5 8 4 5 5 2 9 1]
[3 7 4 9 7 1 7 8 1 5]
24


#### Definir el problema, la función objetivo y las restricciones.

In [3]:
def Problem(values, weights, max_weight):
    
    # initialize a model
    mdl = Model(name="Knaspack")
    
    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(len(values))}  
    # define the objective function
    mdl.minimize(mdl.sum(values[i] * x[i] for i in x))    
    # add the constraints
    mdl.add_constraint(mdl.sum(weights[i] * x[i] for i in x) <= max_weight)
    return  FromDocplex2IsingModel(mdl)

In [4]:
problem =  Problem(values, weights, max_weight)

# Ising encoding of the QUBO problem for binpacking problem
qubo = problem.ising_model

# Docplex encoding of the QUBO problem for binpacking problem
mdl_qubo_docplex = problem.qubo_docplex

mdl_qubo_docplex.prettyprint()

// This file has been generated by DOcplex
// model name is: Knaspack
// var contrainer section
dvar bool slack_C0[5];

// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool x_5;
dvar bool x_6;
dvar bool x_7;
dvar bool x_8;
dvar bool x_9;
dvar bool slack_C0_0;
dvar bool slack_C0_1;
dvar bool slack_C0_2;
dvar bool slack_C0_3;
dvar bool slack_C0_4;

minimize
 - 6766 x_0 - 15787 x_1 - 9019 x_2 - 20296 x_3 - 15788 x_4 - 2251 x_5
 - 15787 x_6 - 18046 x_7 - 2247 x_8 - 11279 x_9 - 2256 slack_C0_0
 - 4512 slack_C0_1 - 9024 slack_C0_2 - 18048 slack_C0_3 - 20304 slack_C0_4 [
 423 x_0^2 + 1974 x_0*x_1 + 1128 x_0*x_2 + 2538 x_0*x_3 + 1974 x_0*x_4
 + 282 x_0*x_5 + 1974 x_0*x_6 + 2256 x_0*x_7 + 282 x_0*x_8 + 1410 x_0*x_9
 + 282 x_0*slack_C0_0 + 564 x_0*slack_C0_1 + 1128 x_0*slack_C0_2
 + 2256 x_0*slack_C0_3 + 2538 x_0*slack_C0_4 + 2303 x_1^2 + 2632 x_1*x_2
 + 5922 x_1*x_3 + 4606 x_1*x_4 + 658 x_1*x_5 + 4606 x_1*x_6 + 5264 x_1*x_7
 + 658 x_1*x_8 +

#### Probar los parámetros para obtener los mejores resultados de la optimización

In [10]:
def funcionesQAOA(device,tipo,ham):
    qaoa = QAOA()
    qaoa.set_device(device)

    # Set the parameters to use the QAOA algorithm
    qaoa.set_backend_properties(n_shots=1024, seed_simulator=1)
    qaoa.set_circuit_properties(p=1, param_type='standard', init_type=tipo, mixer_hamiltonian=ham)

    qaoa.compile(qubo)

    # Run the QAOA algorithm
    qaoa.optimize()
    
    qaoa.cost_hamil
    return qaoa.result

In [11]:
# Initialize the QAOA object and use a device
tipos = ['rand','ramp']
ham = ['xy','x']
dispositivos = ['qiskit.statevector_simulator','qiskit.qasm_simulator']
for k in dispositivos:
    for i in tipos:
        for j in ham:
            device = create_device("local", k)
            results_sv = funcionesQAOA(device,i,j)
            print("Dispositivo: ",k, " Tipo: ",i, " Mixer: ",j)
            print(pd.DataFrame(results_sv.lowest_cost_bitstrings(5)))